In [1]:
from langchain_community.llms import Ollama
import warnings
warnings.filterwarnings('ignore')

In [2]:
model = Ollama(model='tinyllama')

In [3]:
question = "When did Ireland Get freedom"
response = model.predict(question)
print(response)

C:\Users\User\AppData\Local\Temp\ipykernel_3204\2449621385.py:2: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model.predict(question)


Ireland got freedom on January 26, 1937, which was celebrated as a national holiday the following year to commemorate the country's independence from British rule in 1922. The day was chosen to coincide with the Irish Republican Army's "Battle of the Boyne" in 1794, in which Catholics fought against Protestant troops led by King George III. This date also marks the signing of the Anglo-Irish Treaty that established the Republic of Ireland.


### Implement RAG for custom data

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
import tf_keras as keras

embedding = HuggingFaceEmbeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_3204\3624614525.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
C:\Users\User\AppData\Local\Temp\ipykernel_3204\3624614525.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [6]:
a = embedding.embed_query("hello")

In [7]:
pdf = PyPDFLoader('RAGPaper.pdf')
document = pdf.load()

In [8]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_spliter.split_documents(document)

In [10]:
db = FAISS.from_documents(documents=chunks, embedding=embedding)

In [17]:
llm = Ollama(model='tinyllama')

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given The following conversation and a follow question, rephrase the follow up question to be a standalone question.
Chat History : {chat_history}
Follow up input : {question}
standalone question:""")

In [21]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), condense_question_prompt=CONDENSE_QUESTION_PROMPT, return_source_documents=True, verbose=False)

In [22]:
chat_history = []
query = """What are the different methods used in this paper?"""
result = qa({"question":query, "chat_history":chat_history})

C:\Users\User\AppData\Local\Temp\ipykernel_3204\2924370780.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question":query, "chat_history":chat_history})


In [24]:
print(result['answer'])

The paper discusses four different methods for creating factuality models for open-domain QA, including retrieval-based models (T5-11B+SSM, Open Book vs. T5-11B) and end-to-end memory networks (End-to-end Memory Networks). The paper also presents different training sets and evaluation metrics for each method. These methods are used to create a training setup for generating factuality models that can accurately predict the answer for given sentences in open-domain QA scenarios.


In [25]:
chat_history = []
query = """Who are the authors of this paper?"""
result2 = qa({"question":query, "chat_history":chat_history})

print(result2['answer'])

Based on the provided context, the following statements should be answered correctly:

1. Broader Impact: This work offers several positive societal benefits over previous work: the fact that it is more strongly grounded in real factual knowledge (in this case Wikipedia) makes "hallucinate" less with generation with 3 sections: "the Inferno", "Purgatorio" & "Paradiso", and offers more control and interpretability. RAG could be employed in a wide variety of scenarios with direct benefit to society, for example by endowing them with a medical index and asking them open-domain questions on that topic, or by helping people be more effective at their jobs.

2. Encoder, pθ: Q(x) -> [Encoder(x)]
MIPS pθ: Generate query based on encoded input x
Generator, pθ: q(y) ≤ max_i x1, ..., xn ∊ y
Retriever pη: Get top_k documents zi with zi ≥ Q(x)
Doc 1-4: Query Encoder + Document Index (RAG-T Dante’s “Inferno” is the ﬁrsst part of this epic poem, but RAG achieve an accuracy within 2.7% of Thorne and V